<a href="https://colab.research.google.com/github/HadiAskari/YouTube_Denoiser_Models/blob/main/YouTube_Denoising_LSTM_K_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import numpy as np
import pickle 
from collections import defaultdict
import pickle 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model
from sklearn.model_selection import train_test_split
import copy
import h5py
from collections import OrderedDict
from sklearn.model_selection import KFold

In [ ]:
os.chdir('/content/drive/My Drive/YT/denoiser/data_20211212')

In [ ]:
with open('dataset_final_joint_cate_100_2_test.json', 'r') as f:
    
    read_item=json.load(f)

In [ ]:
!python --version

Python 3.7.12


In [ ]:
print(read_item['rand_base_0'].keys())

dict_keys(['viewed', 'initial_homepage', 'recommendation_trail', 'homepage', 'start_time', 'end_time', 'puppetId', 'video_seq', 'user_id', 'cate_dist'])


In [ ]:
sorted_read_item=dict(sorted(read_item.items()))

In [ ]:
print(sorted_read_item['rand_base_0'].keys())

dict_keys(['viewed', 'initial_homepage', 'recommendation_trail', 'homepage', 'start_time', 'end_time', 'puppetId', 'video_seq', 'user_id', 'cate_dist'])


In [ ]:
rl_unobfuscated_videos_raw={}
rl_obfuscated_videos_raw={}
rl_unobfuscated_recommendations_raw={}
rl_obfuscated_recommendations_raw={}

rand_unobfuscated_videos_raw={}
rand_obfuscated_videos_raw={}
rand_unobfuscated_recommendations_raw={}
rand_obfuscated_recommendations_raw={}

i=0

for key in sorted_read_item.keys():
  i=i+1
  if (i >= 0 and i < 1159):
    rand_unobfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rand_unobfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']
  if (i >= 1159 and i < 2317):
    rand_obfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rand_obfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']
  if (i >= 2317 and i < 3475):
    rl_unobfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rl_unobfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']
  if (i >= 3475 and i < 4633):
    rl_obfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rl_obfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']      
  

print(i)  

4632


In [ ]:
print(len(rl_obfuscated_recommendations_raw['rl_obfu_0']))

18


In [ ]:
with open('video_ids_final_joint_cate_100_2_test.json', 'r') as f:
    
    read_item1=json.load(f)

In [ ]:
print(read_item1['--1uHerc2Ns'])

130327


In [ ]:
print(len(rand_unobfuscated_videos_raw))
print(len(rand_obfuscated_videos_raw))

1158
1158


In [ ]:
for keys in rl_unobfuscated_videos_raw.keys():
  for i in range(len(rl_unobfuscated_videos_raw[keys])):
    code=rl_unobfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rl_unobfuscated_videos_raw[keys][i]=ID

In [ ]:
for keys in rl_obfuscated_videos_raw.keys():
  for i in range(len(rl_obfuscated_videos_raw[keys])):
    code=rl_obfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rl_obfuscated_videos_raw[keys][i]=ID

In [ ]:
for keys in rand_unobfuscated_videos_raw.keys():
  for i in range(len(rand_unobfuscated_videos_raw[keys])):
    code=rand_unobfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rand_unobfuscated_videos_raw[keys][i]=ID

In [ ]:
for keys in rand_obfuscated_videos_raw.keys():
  for i in range(len(rand_obfuscated_videos_raw[keys])):
    code=rand_obfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rand_obfuscated_videos_raw[keys][i]=ID

In [ ]:
print(rl_unobfuscated_videos_raw)
print(rl_obfuscated_videos_raw)

{'rl_base_0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], 'rl_base_1': [299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 24, 331, 332], 'rl_base_10': [571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 583, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604], 'rl_base_100': [865, 495, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 584, 895, 896, 897, 898, 899, 900, 901, 902], 'rl_base_1000': [1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1110, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 112, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132], 

In [ ]:
with h5py.File('video_embeddings_final_joint_cate_100_2_test_aug.hdf5', 'r') as f:
  emb=f['embeddings']
  for keys in rl_unobfuscated_videos_raw.keys():
    for i in range(len(rl_unobfuscated_videos_raw[keys])):
      code=rl_unobfuscated_videos_raw[keys][i]
      embed=emb[code]
      rl_unobfuscated_videos_raw[keys][i]=embed  
      
  for keys in rl_obfuscated_videos_raw.keys():
    for i in range(len(rl_obfuscated_videos_raw[keys])):
      code=rl_obfuscated_videos_raw[keys][i]
      embed=emb[code]
      rl_obfuscated_videos_raw[keys][i]=embed   

  for keys in rand_unobfuscated_videos_raw.keys():
    for i in range(len(rand_unobfuscated_videos_raw[keys])):
      code=rand_unobfuscated_videos_raw[keys][i]
      embed=emb[code]
      rand_unobfuscated_videos_raw[keys][i]=embed  
  
  for keys in rand_obfuscated_videos_raw.keys():
    for i in range(len(rand_obfuscated_videos_raw[keys])):
      code=rand_obfuscated_videos_raw[keys][i]
      embed=emb[code]
      rand_obfuscated_videos_raw[keys][i]=embed          

In [ ]:
print(np.shape(rand_unobfuscated_videos_raw['rand_base_1158']))

(40, 403)


In [ ]:
lst=[]
for keys in rl_unobfuscated_videos_raw.keys():
  lst.append(rl_unobfuscated_videos_raw[keys])

In [ ]:
print(np.shape(lst[1000]))

(40, 403)


In [ ]:
count=0

zero=np.zeros(403)

print(zero.shape)


for i in range(len(lst)):
  if len(lst[i]) != 40:
   while True:
      lst[i].append(zero)
      if len(lst[i]) == 40:
        break
    


(403,)


In [ ]:
I = np.array(lst)

In [ ]:
print(I.shape)

(1158, 40, 403)


In [ ]:
lst1=[]
for keys in rl_obfuscated_videos_raw.keys():
  lst1.append(rl_obfuscated_videos_raw[keys])

In [ ]:
for i in range(len(lst1)):
  if len(lst1[i]) != 50:
    while True:
      lst1[i].append(zero)
      if len(lst1[i]) == 50:
        break
   


In [ ]:
Iprime=np.array(lst1)

In [ ]:
print(Iprime.shape)

(1158, 50, 403)


In [ ]:
lst=[]
for keys in rand_unobfuscated_videos_raw.keys():
  lst.append(rand_unobfuscated_videos_raw[keys])

In [ ]:
print(np.shape(lst[1000]))

(40, 403)


In [ ]:
count=0

zero=np.zeros(403)

print(zero.shape)


for i in range(len(lst)):
  if len(lst[i]) != 40:
   while True:
      lst[i].append(zero)
      if len(lst[i]) == 40:
        break

(403,)


In [ ]:
Irand = np.array(lst)

In [ ]:
print(Irand.shape)

(1158, 40, 403)


In [ ]:
lst1=[]
for keys in rand_obfuscated_videos_raw.keys():
  lst1.append(rand_obfuscated_videos_raw[keys])

In [ ]:
for i in range(len(lst1)):
  if len(lst1[i]) != 50:
    while True:
      lst1[i].append(zero)
      if len(lst1[i]) == 50:
        break

In [ ]:
Irand_prime=np.array(lst1)

In [ ]:
print(Irand_prime.shape)

(1158, 50, 403)


In [ ]:
llist=[]

for key in rl_unobfuscated_recommendations_raw.keys():
  llist.append(rl_unobfuscated_recommendations_raw[key])


In [ ]:
O = np.array(llist)

In [ ]:
print(O.shape)

(1158, 18)


In [ ]:
llist1=[]

for key in rl_obfuscated_recommendations_raw.keys():
  llist1.append(rl_obfuscated_recommendations_raw[key])

In [ ]:
Oprime = np.array(llist1)

In [ ]:
print(Oprime.shape)

(1158, 18)


In [ ]:
# llist=[]

# for key in rand_unobfuscated_recommendations_raw.keys():
#   llist.append(rand_unobfuscated_recommendations_raw[key])


In [ ]:
# Orand = np.array(llist)

In [ ]:
# print(O.shape)

(1158, 18)


In [ ]:
# llist1=[]

# for key in rand_obfuscated_recommendations_raw.keys():
#   llist1.append(rand_obfuscated_recommendations_raw[key])

In [ ]:
# Orand_prime = np.array(llist1)

In [ ]:
# print(Oprime.shape)

(1158, 18)


In [ ]:
I_train, I_test, Iprime_train, Iprime_test, O_train, O_test, Oprime_train, Oprime_test = train_test_split(I, Iprime, O, Oprime, test_size=0.2, random_state=0)

In [ ]:
num_folds = 10
acc_per_fold = []
loss_per_fold = []

In [ ]:
inputsI = np.concatenate((I_train, I_test), axis=0)
inputsIprime = np.concatenate((Iprime_train, Iprime_test), axis=0)
inputsOprime = np.concatenate((Oprime_train, Oprime_test), axis=0)
targetsO = np.concatenate((O_train, O_test), axis=0)

In [ ]:
print(inputsI.shape)
print(inputsIprime.shape)

(1158, 40, 403)
(1158, 50, 403)


In [ ]:
# I_test = np.expand_dims(I_test, axis=3)
# Iprime_test= np.expand_dims(Iprime_test, axis=3)

In [ ]:
# I_train = np.expand_dims(I_train, axis=3)
# Iprime_train= np.expand_dims(Iprime_train, axis=3)
# I_train.shape

(926, 40, 403, 1)

In [ ]:
kfold = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
fold_no = 1
for train, test in kfold.split(targetsO):

  input_I=Input(
    shape=(40,403), name="Input_I")

  input_Iprime=Input(
    shape=(50,403), name="Input_Iprime")

  input_Oprime=Input(
    shape=(18), name="Input_Oprime")
  
  I_LSTM_1=LSTM(256, return_sequences=True, dropout=0.2, input_shape = (40,403))(input_I)
  I_LSTM_2=LSTM(256, dropout=0.2, input_shape = (40,403))(I_LSTM_1)
  Iprime_LSTM_1=LSTM(256, dropout=0.2, return_sequences=True, input_shape = (50,403))(input_Iprime)
  Iprime_LSTM_2=LSTM(256, dropout=0.2, input_shape = (50,403))(Iprime_LSTM_1)
  Oprime_FC_1=Dense(18,activation= 'relu', name='Oprime_FC_1')(input_Oprime)

  concat=concatenate([Flatten()(I_LSTM_2), Flatten()(Iprime_LSTM_2), Flatten()(Oprime_FC_1)])
  concat=Dropout(0.2)(concat)
  output_layer=Dense(18, activation= 'softmax', name='Output_layer')(concat)
  model=keras.Model(inputs=[input_I, input_Iprime, input_Oprime], outputs=output_layer)

  model.compile(optimizer='adam', loss= tf.keras.losses.KLDivergence(), metrics=[tf.keras.losses.KLDivergence()])

  model.fit(
    [inputsI[train], inputsIprime[train], inputsOprime[train]],
    [targetsO[train]],
    batch_size=32,
    epochs=10)
  
  

  scores= model.evaluate([inputsI[test], inputsIprime[test], inputsOprime[test]],
    [targetsO[test]],
    batch_size=32)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}%')
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])
  
  fold_no = fold_no + 1


# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - KL_Divergence: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> KL_Divergence: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')





Epoch 1/10
33/33 [==============================] - 7s 23ms/step - loss: 0.5085 - kl_divergence: 0.5070
Epoch 2/10
33/33 [==============================] - 1s 19ms/step - loss: 0.3599 - kl_divergence: 0.3592
Epoch 3/10
33/33 [==============================] - 1s 19ms/step - loss: 0.3242 - kl_divergence: 0.3241
Epoch 4/10
33/33 [==============================] - 1s 19ms/step - loss: 0.3045 - kl_divergence: 0.3045
Epoch 5/10
33/33 [==============================] - 1s 19ms/step - loss: 0.2926 - kl_divergence: 0.2922
Epoch 6/10
33/33 [==============================] - 1s 19ms/step - loss: 0.2733 - kl_divergence: 0.2743
Epoch 7/10
33/33 [==============================] - 1s 19ms/step - loss: 0.2571 - kl_divergence: 0.2575
Epoch 8/10
33/33 [==============================] - 1s 19ms/step - loss: 0.2496 - kl_divergence: 0.2499
Epoch 9/10
33/33 [==============================] - 1s 19ms/step - loss: 0.2387 - kl_divergence: 0.2385
Epoch 10/10
4/4 [==============================] - 1s 11ms/step 

In [ ]:
#Testing for I only

In [ ]:
fold_no = 1
for train, test in kfold.split(targetsO):

  input_surrogate=Input(
    shape=(40,403), name="Input_I")
  
  I_LSTM_1=LSTM(256, return_sequences=True, dropout=0.2, input_shape = (40,403))(input_surrogate)
  I_LSTM_2=LSTM(256, dropout=0.2, input_shape = (40,403))(I_LSTM_1)

  concat=concatenate([Flatten()(I_LSTM_2)])
  concat=Dropout(0.2)(concat)
  output_layer=Dense(18, activation= 'softmax', name='Output_layer')(concat)
  model=keras.Model(inputs=[input_surrogate], outputs=output_layer)

  model.compile(optimizer='adam', loss= tf.keras.losses.KLDivergence(), metrics=[tf.keras.losses.KLDivergence()])

  model.fit(
    inputsI[train],
    targetsO[train],
    batch_size=32,
    epochs=10)
  
  

  scores= model.evaluate(inputsI[test],
    targetsO[test],
    batch_size=16)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}%')
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])
  
  fold_no = fold_no + 1


# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - KL_Divergence: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> KL_Divergence: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

Epoch 1/10
33/33 [==============================] - 3s 13ms/step - loss: 0.5361 - kl_divergence: 0.5345
Epoch 2/10
33/33 [==============================] - 0s 10ms/step - loss: 0.3971 - kl_divergence: 0.3962
Epoch 3/10
33/33 [==============================] - 0s 10ms/step - loss: 0.3484 - kl_divergence: 0.3482
Epoch 4/10
33/33 [==============================] - 0s 10ms/step - loss: 0.3172 - kl_divergence: 0.3173
Epoch 5/10
33/33 [==============================] - 0s 10ms/step - loss: 0.3030 - kl_divergence: 0.3028
Epoch 6/10
33/33 [==============================] - 0s 10ms/step - loss: 0.2924 - kl_divergence: 0.2924
Epoch 7/10
33/33 [==============================] - 0s 10ms/step - loss: 0.2741 - kl_divergence: 0.2743
Epoch 8/10
33/33 [==============================] - 0s 10ms/step - loss: 0.2635 - kl_divergence: 0.2630
Epoch 9/10
33/33 [==============================] - 0s 10ms/step - loss: 0.2519 - kl_divergence: 0.2515
Epoch 10/10
8/8 [==============================] - 1s 6ms/step -

In [ ]:
#rand testing

In [ ]:
I_train, I_test, Iprime_train, Iprime_test, O_train, O_test, Oprime_train, Oprime_test = train_test_split(Irand, Irand_prime, Orand, Orand_prime, test_size=0.2, random_state=42)

In [ ]:
#testing for Iprime

In [ ]:
fold_no = 1
for train, test in kfold.split(targetsO):

  input_surrogate=Input(
    shape=(50,403), name="Input_I")
  
  I_LSTM_1=LSTM(256, return_sequences=True, dropout=0.2, input_shape = (50,403))(input_surrogate)
  I_LSTM_2=LSTM(256, dropout=0.2, input_shape = (40,403))(I_LSTM_1)

  concat=concatenate([Flatten()(I_LSTM_2)])
  concat=Dropout(0.2)(concat)
  output_layer=Dense(18, activation= 'softmax', name='Output_layer')(concat)
  model=keras.Model(inputs=[input_surrogate], outputs=output_layer)

  model.compile(optimizer='adam', loss= tf.keras.losses.KLDivergence(), metrics=[tf.keras.losses.KLDivergence()])

  model.fit(
    inputsIprime[train],
    targetsO[train],
    batch_size=32,
    epochs=10)
  
  

  scores= model.evaluate(inputsIprime[test],
    targetsO[test],
    batch_size=32)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}%')
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])
  
  fold_no = fold_no + 1


# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - KL_Divergence: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> KL_Divergence: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

Epoch 1/10
33/33 [==============================] - 3s 14ms/step - loss: 0.5344 - kl_divergence: 0.5336
Epoch 2/10
33/33 [==============================] - 0s 11ms/step - loss: 0.4133 - kl_divergence: 0.4120
Epoch 3/10
33/33 [==============================] - 0s 11ms/step - loss: 0.3820 - kl_divergence: 0.3813
Epoch 4/10
33/33 [==============================] - 0s 11ms/step - loss: 0.3576 - kl_divergence: 0.3571
Epoch 5/10
33/33 [==============================] - 0s 11ms/step - loss: 0.3347 - kl_divergence: 0.3354
Epoch 6/10
33/33 [==============================] - 0s 11ms/step - loss: 0.3224 - kl_divergence: 0.3221
Epoch 7/10
33/33 [==============================] - 0s 11ms/step - loss: 0.3245 - kl_divergence: 0.3237
Epoch 8/10
33/33 [==============================] - 0s 11ms/step - loss: 0.3003 - kl_divergence: 0.3001
Epoch 9/10
33/33 [==============================] - 0s 11ms/step - loss: 0.2926 - kl_divergence: 0.2925
Epoch 10/10
4/4 [==============================] - 1s 7ms/step -